In [1]:
!pip install --upgrade langchain langchain-community langgraph langchain-google-genai  wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 21.5 MB/s eta 0:00:00
  Created wheel

# Set LLM

In [2]:
import os

from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


# Tool Building

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki_tools = WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
wiki_tools({"query":"AI"})

'Page: Artificial intelligence\nSummary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.\nHigh-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general appli

In [8]:
tools = [wiki_tools]

#tool binding
llm_with_tools = llm.bind_tools(tools)

#tool calling
result = llm_with_tools.invoke("hello ?")
result.content

'Hello! How can I help you today?'

# Agent Building

In [16]:
from langgraph.prebuilt import create_react_agent

# define agent
agent_executor = create_react_agent(llm, tools)

In [17]:
from langchain_core.messages import HumanMessage

# test agent
response = agent_executor.invoke({"messages" :[HumanMessage(content="hello ?")]})

response["messages"]

[HumanMessage(content='hello ?', additional_kwargs={}, response_metadata={}, id='fae7878d-ce09-4065-a2be-7eae2275d809'),
 AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f531b3fc-cbad-4e71-a1ec-7d21283d6bc5-0', usage_metadata={'input_tokens': 50, 'output_tokens': 10, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}})]

In [19]:
# print only the output
print(response["messages"][-1].content)

Hello! How can I help you today?


In [21]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages" :[HumanMessage(content="What is Agentic AI")]})

response["messages"]

[HumanMessage(content='What is Agentic AI', additional_kwargs={}, response_metadata={}, id='a93b3503-13bd-44ac-a1e1-62865c1aa448'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'wikipedia', 'arguments': '{"query": "Agentic AI"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--84a237fb-fad9-420c-806b-2fae8fde6d10-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Agentic AI'}, 'id': '5944f128-191c-4e61-a9d2-6ffa0c431506', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 5, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='Page: Agentic AI\nSummary: Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce p

In [22]:
# tool output
print(response["messages"][-2].content)

Page: Agentic AI
Summary: Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence.


In [23]:
# agent output
print(response["messages"][-1].content)

Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence.
